In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference, MetricFrame
from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.reductions import ExponentiatedGradient, DemographicParity

In [3]:
df = pd.read_csv('C:/Users/User/Downloads/archive/job_applicant_dataset.csv')

In [4]:
df.drop(columns=['Resume', 'Job Description'], axis=1, inplace=True)

df.head()

,Job Applicant Name,Age,Gender,Race,Ethnicity,Job Roles,Best Match
0,Daisuke Mori,29,Male,Mongoloid/Asian,Vietnamese,Fitness Coach,0
1,Taichi Shimizu,31,Male,Mongoloid/Asian,Filipino,Physician,0
2,Sarah Martin,46,Female,White/Caucasian,Dutch,Financial Analyst,0
3,Keith Hughes,43,Male,Negroid/Black,Caribbean,Supply Chain Manager,1
4,James Davis,49,Male,White/Caucasian,English,Supply Chain Manager,1


In [5]:
x = df.drop(columns=['Best Match', 'Gender'], axis=1)
y = df['Best Match']
sensitive_feature = df['Gender']

In [6]:
x = pd.get_dummies(x, columns=['Job Applicant Name', 'Race', 'Ethnicity', 'Job Roles'], drop_first=True).astype(int)

In [7]:
x_train, x_test, y_train, y_test, sens_train, sens_test = train_test_split(
    x, y, sensitive_feature, test_size=.20, random_state=42
)

In [8]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [9]:
y_pred = model.predict(x_test)
y_pred_proba = model.predict_proba(x_test)[:, 1]

1. Check Demographic Parity
- Measures: Are positive predictions distributed equally?

```
0.0 = perfect fairness, 1.0 = maximum bias
Should be < 0.1 for fairness 
```

In [10]:
dp_diff = demographic_parity_difference(
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sens_test
)

print(f"Demographic Parity Difference: {dp_diff:.3f}")

Demographic Parity Difference: 0.404


2. Check Equalized Odds
- Measures: Are TPR and FPR equal across groups?
``` 
Should be < 0.1 for fairness

In [11]:
eo_diff = equalized_odds_difference(
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sens_test
)

print(f"Equalized Odds Difference: {eo_diff:.3f}")

Equalized Odds Difference: 0.386


3. Detailed Metrics by Group

In [12]:
metrics = {
    'accuracy': accuracy_score,
    'selection_rate': lambda y_true, y_pred: y_pred.mean()
}

metric_frame = MetricFrame(
    metrics=metrics,
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sens_test
)

print("\nMetrics by Group:")
print(metric_frame.by_group)

# Check for disparities
print("\nDisparities:")
print(metric_frame.difference())


Metrics by Group:
        accuracy  selection_rate
Gender                          
Female  0.615067        0.241486
Male    0.580019        0.645005

Disparities:
accuracy          0.035048
selection_rate    0.403519
dtype: float64


##### Mitigating Bias
1. Post-Processing: Threshold Optimization

In [13]:
mitigator = ThresholdOptimizer(
    estimator=model,
    constraints='demographic_parity'
)

mitigator.fit(x_train, y_train, sensitive_features=sens_train)

,estimator,RandomForestC...ndom_state=42)
,constraints,'demographic_parity'
,objective,'accuracy_score'
,grid_size,1000
,flip,False
,prefit,False
,predict_method,'auto'
,tol,None
,n_estimators,100
,criterion,'gini'
,max_depth,None


In [14]:
# Predict with fairness constraints
fair_predictions = mitigator.predict(x_test, sensitive_features=sens_test)

In [15]:
# Check improved fairness
dp_diff_fair = demographic_parity_difference(
    y_true=y_test,
    y_pred=fair_predictions,
    sensitive_features=sens_test
)
print(f"Demographic Parity (after mitigation): {dp_diff_fair:.3f}")

Demographic Parity (after mitigation): 0.416


2. In-Processing: Fairness-Aware Training

In [16]:
# Use exponentiated gradient for fairness-aware training
constraint = DemographicParity()

mitigator = ExponentiatedGradient(
    estimator=RandomForestClassifier(n_estimators=100),
    constraints=constraint
)

mitigator.fit(x_train, y_train, sensitive_features=sens_train)
fair_predictions = mitigator.predict(x_test)

In [17]:
# Verify fairness
dp_diff = demographic_parity_difference(
    y_true=y_test,
    y_pred=fair_predictions,
    sensitive_features=sens_test
)
print(f"Demographic Parity: {dp_diff:.3f}")

Demographic Parity: 0.070


#### Project: Audit Titanic Survival Model

In [58]:
df = pd.read_csv('https://raw.githubusercontent.com/mdmohsin212/Machine-Learning/refs/heads/main/dataset/Titanic.csv')

In [59]:
from sklearn.preprocessing import LabelEncoder

df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'])

lb = LabelEncoder()
df['Sex'] = lb.fit_transform(df['Sex'])

df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [60]:
df['Age'] = df['Age'].fillna(df['Age'].mean())

df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
dtype: int64

In [61]:
x = df[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']]
y = df['Survived']
sensitive_feature = df['Sex']

In [62]:
x_train, x_test, y_train, y_test, sens_train, sens_test = train_test_split(
    x, y, sensitive_feature, test_size=.20, random_state=42
)

In [63]:
model = RandomForestClassifier(n_estimators=101, random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

In [64]:
# Audit for gender bias
dp_diff = demographic_parity_difference(
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sens_test
)
print(f"Gender Bias (Demographic Parity): {dp_diff:.3f}")

if dp_diff > 0.1:
    print("WARNING: Model shows significant gender bias!")
    print("Consider using ThresholdOptimizer to mitigate bias.")


Gender Bias (Demographic Parity): 0.253
Consider using ThresholdOptimizer to mitigate bias.


In [65]:
# Check by passenger class
sensitive_feature_check = df.loc[y_test.index, 'Pclass']
dp_diff_class = demographic_parity_difference(
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sens_test
)

print(f"\nClass Bias (Demographic Parity): {dp_diff_class:.3f}")


Class Bias (Demographic Parity): 0.253


**Additional Tools -> AIF360 (IBM)**

In [66]:
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

In [67]:
# Convert to AIF360 format
dataset = BinaryLabelDataset(
    df=df,
    label_names=['Survived'],
    protected_attribute_names=['Sex'],
    favorable_label=1,
    unfavorable_label=0
)

In [70]:
metric = BinaryLabelDatasetMetric(
    dataset,
    unprivileged_groups=[{'Sex': 1}],
    privileged_groups=[{'Sex': 0}]
)

print(f"Disparate Impact: {metric.disparate_impact()}")

Disparate Impact: 0.2545800760184765
